# FEMA county-level data
Is Federal Emergency Management Agency a relevant source of county-level data relevant to soil health and watershed function, say on expenditures for flood, drought?

FEMA dates from 1979, as a consolidation of other federal disaster-relief agencies. In 2003 it became part of Homeland Security.


This is the main URL from which you can navigate to the various datasets:

https://www.fema.gov/openfema

FEMA spends money in basically THREE WAYS:

1. Public assistance (PA): grants to local governments and organizations
2. Individual assistance (IA): grants to homeowners and renters
3. Hazard mitigation (HM) a program begun in 1989

In order to get county-level expenditures by year and by disaster type (such as Flood, Drought, Mudslide), it will be necessary to aggregate data from the three categories, or find already aggregated summaries.

RESULT WANTED: map FEMA expenditures by county and year from around 1980 to present, at least for soil-health-related categories of disaster such as flood, drought, landslide, levee break. While not all flood or drought disasters can be "blamed" on soil degradation or poor soil stewardship, there is a relation here that can be local and is often overlooked or not recognized. The costs of disaster mitigation can be understood as an *economic and social opportunity: to rebuild the soil carbon sponge, restore hydrologic function, and possibly even reduce the severity of storms and cool the earth.*

The available data sets are confusing and incomplete. Several hours of looking at available files and APIs seems to show that for the financial data, HM figures are available from 1989, PA figures from 1998, and IA figures from 2004, and emails from the openfema team confirm this. Location data is available in some form from 1953. (Since financial data is so spotty, one map could be the NUMBER of disaster declarations by county, or time series by year of declaration. In fact this map is already done by FEMA, I've seen it somewhere.)

What looks to be the best strategy is a large joined table, with disaster number as a join field, and keeping location fields from several tables as a cross-check.

Disaster numbers can have several counties as location, and some PA grants are statewide.

Strategy?: 1. filter for soil-related, to get lists of disaster numbers with dates of declaration.
           2. group and sum expenditures by location (standardize groups to FIPS, list of FIPS, or state code?)
           3. inner joins with soil-related disaster numbers while preserving source filenames or suffixes for cols; do some cross-checking if possible


In [1]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize

## Finding the right files

can be a challenge, you just have to look at each one and try to figure it out.

In general, location information and financial information are separate.

The most complete file for disaster number, location, and date appears to be an Excel sheet from

https://www.fema.gov/media-library/assets/documents/28318


# Getting location
data.gov.FEMADeclarations.6.11.18.xlsx is the most complete for disaster numbers, and for most it has place codes and locations. The place codes are in the FEMA format of 99 plus the three-digit county FIPS codes.

First we'll filter for soil-related disasters.

In [2]:
df = pd.read_excel('fema/comb/data.gov.FEMADeclarations.6.11.18.xlsx', 'FEMA Declarations', skiprows=0) #with original file may need to use skiprows=2

In [3]:
soil = ['Severe Storm(s)', 'Mud/Landslide', 'Flood','Dam/Levee Break', 'Drought','Fire']
df = df[df['Incident Type'].isin(soil)]
df['Incident Type'].unique()


array(['Flood', 'Fire', 'Severe Storm(s)', 'Drought', 'Dam/Levee Break',
       'Mud/Landslide'], dtype=object)

In [4]:
#FIPS CODE stuff. We need FIPS codes for counties. FEMA uses 99 as prefix for 3-digit county codes.
#convert Place Code to string
df['placeCode'] = df['Place Code'].map(lambda x: str(x))

In [5]:
st = pd.read_csv('stateFIPS.csv')
stfips = st.set_index('code').to_dict()['FIPS']
print(stfips)

{'AL': 1, 'AK': 2, 'AZ': 4, 'AR': 5, 'CA': 6, 'CO': 8, 'CT': 9, 'DE': 10, 'DC': 11, 'FL': 12, 'GA': 13, 'HI': 15, 'ID': 16, 'IL': 17, 'IN': 18, 'IA': 19, 'KS': 20, 'KY': 21, 'LA': 22, 'ME': 23, 'MD': 24, 'MA': 25, 'MI': 26, 'MN': 27, 'MS': 28, 'MO': 29, 'MT': 30, 'NE': 31, 'NV': 32, 'NH': 33, 'NJ': 34, 'NM': 35, 'NY': 36, 'NC': 37, 'ND': 38, 'OH': 39, 'OK': 40, 'OR': 41, 'PA': 42, 'RI': 44, 'SC': 45, 'SD': 46, 'TN': 47, 'TX': 48, 'UT': 49, 'VT': 50, 'VA': 51, 'WA': 53, 'WV': 54, 'WI': 55, 'WY': 56, 'AS': 60, 'FM': 64, 'GU': 66, 'MH': 68, 'MP': 69, 'PW': 70, 'PR': 72, 'UM': 74, 'VI': 78}


In [6]:
df['stFIPS'] = df['State '].map(stfips)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29617 entries, 2 to 47913
Data columns (total 17 columns):
Disaster Number            29617 non-null int64
IH Program Declared        29617 non-null object
IA Program Declared        29617 non-null object
PA Program Declared        29617 non-null object
HM Program Declared        29617 non-null object
State                      29617 non-null object
Declaration Date           29617 non-null datetime64[ns]
Disaster Type              29617 non-null object
Incident Type              29617 non-null object
Title                      29617 non-null object
Incident Begin Date        29617 non-null datetime64[ns]
Incident End Date          29275 non-null datetime64[ns]
Disaster Close Out Date    23288 non-null datetime64[ns]
Place Code                 29486 non-null float64
Declared County/Area       29486 non-null object
placeCode                  29617 non-null object
stFIPS                     29617 non-null int64
dtypes: datetime64[ns](4), 

In [10]:

#df[df['Declared County/Area']=='Valencia County']
#df.iloc[372]['Place Code']

,Disaster Number,IH Program Declared,IA Program Declared,PA Program Declared,HM Program Declared,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date,Place Code,Declared County/Area,placeCode,stFIPS


In [49]:
df['Place Code'] = df['Place Code'].fillna(value=0)

df['Place Code'] = df['Place Code'].astype(int)
df.dtypes

Disaster Number                     int64
IH Program Declared                object
IA Program Declared                object
PA Program Declared                object
HM Program Declared                object
State                              object
Declaration Date           datetime64[ns]
Disaster Type                      object
Incident Type                      object
Title                              object
Incident Begin Date        datetime64[ns]
Incident End Date          datetime64[ns]
Disaster Close Out Date    datetime64[ns]
Place Code                          int32
Declared County/Area               object
placeCode                          object
stFIPS                              int64
dtype: object

In [9]:
df.dtypes

Disaster Number                     int64
IH Program Declared                object
IA Program Declared                object
PA Program Declared                object
HM Program Declared                object
State                              object
Declaration Date           datetime64[ns]
Disaster Type                      object
Incident Type                      object
Title                              object
Incident Begin Date        datetime64[ns]
Incident End Date          datetime64[ns]
Disaster Close Out Date    datetime64[ns]
Place Code                        float64
Declared County/Area               object
placeCode                          object
stFIPS                              int64
dtype: object

In [15]:
df.dtypes

Disaster Number                     int64
IH Program Declared                object
IA Program Declared                object
PA Program Declared                object
HM Program Declared                object
State                              object
Declaration Date           datetime64[ns]
Disaster Type                      object
Incident Type                      object
Title                              object
Incident Begin Date        datetime64[ns]
Incident End Date          datetime64[ns]
Disaster Close Out Date    datetime64[ns]
Place Code                         object
Declared County/Area               object
placeCode                          object
stFIPS                              int64
dtype: object

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29617 entries, 2 to 47913
Data columns (total 17 columns):
Disaster Number            29617 non-null int64
IH Program Declared        29617 non-null object
IA Program Declared        29617 non-null object
PA Program Declared        29617 non-null object
HM Program Declared        29617 non-null object
State                      29617 non-null object
Declaration Date           29617 non-null datetime64[ns]
Disaster Type              29617 non-null object
Incident Type              29617 non-null object
Title                      29617 non-null object
Incident Begin Date        29617 non-null datetime64[ns]
Incident End Date          29275 non-null datetime64[ns]
Disaster Close Out Date    23288 non-null datetime64[ns]
Place Code                 29617 non-null int32
Declared County/Area       29486 non-null object
placeCode                  29617 non-null object
stFIPS                     29617 non-null int64
dtypes: datetime64[ns](4), in

In [60]:
print(type(df.iloc[372]['Place Code']))

<class 'numpy.int32'>


In [63]:
df['Place Code']

2            0
4            0
7            0
8            0
9            0
10           0
13           0
15           0
16           0
18           0
19           0
25           0
28           0
31           0
33           0
34           0
36           0
38           0
45           0
46           0
47           0
48           0
49           0
50           0
53           0
54           0
56           0
57           0
58           0
59           0
         ...  
47882    99073
47883    99077
47884    99085
47885    99089
47886    99091
47887    99099
47888    99111
47889    99115
47890    99123
47891    99127
47892    99131
47893    99141
47894    99147
47895    99149
47896    99155
47897    99163
47898    99165
47899    99169
47900    99171
47901    99173
47902    99181
47903    99081
47904    99157
47905    99003
47906    99007
47909    99003
47910    99013
47911    99027
47912    99031
47913    99035
Name: Place Code, dtype: int32

In [64]:
#NEED TO FIND WAY TO GET FIPS!!
for row in df.iterrows():
    if row['Place Code'][0:2]=='99':
        row['coFIPS'] = row['Place Code'][2:6]
            

TypeError: tuple indices must be integers or slices, not str

In [80]:
df

,Disaster Number,IH Program Declared,IA Program Declared,PA Program Declared,HM Program Declared,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date,Place Code,Declared County/Area,stFIPS,FIPS
2,3,No,Yes,Yes,Yes,LA,1953-05-29 00:00:00,DR,Flood,FLOOD,1953-05-29 00:00:00,1953-05-29 00:00:00,1960-02-01,0,NaN,22,
4,5,No,Yes,Yes,Yes,MT,1953-06-06 00:00:00,DR,Flood,FLOODS,1953-06-06 00:00:00,1953-06-06 00:00:00,1955-12-01,0,NaN,30,
7,8,No,Yes,Yes,Yes,IA,1953-06-11 00:00:00,DR,Flood,FLOOD,1953-06-11 00:00:00,1953-06-11 00:00:00,1955-11-01,0,NaN,19,
8,9,No,Yes,Yes,Yes,TX,1953-06-19 00:00:00,DR,Flood,FLOOD,1953-06-19 00:00:00,1953-06-19 00:00:00,1958-01-01,0,NaN,48,
9,11,No,Yes,Yes,Yes,NH,1953-07-02 00:00:00,DR,Fire,FOREST FIRE,1953-07-02 00:00:00,1953-07-02 00:00:00,1956-02-01,0,NaN,33,
10,12,No,Yes,Yes,Yes,FL,1953-10-22 00:00:00,DR,Flood,FLOOD,1953-10-22 00:00:00,1953-10-22 00:00:00,1956-05-01,0,NaN,12,
13,15,No,Yes,Yes,Yes,CA,1954-02-05 00:00:00,DR,Flood,FLOOD & EROSION,1954-02-05 00:00:00,1954-02-05 00:00:00,1957-09-01,0,NaN,6,
15,17,No,Yes,Yes,Yes,IA,1954-06-23 00:00:00,DR,Flood,FLOOD,1954-06-23 00:00:00,1954-06-23 00:00:00,1956-02-01,0,NaN,19,
16,18,No,Yes,Yes,Yes,TX,1954-07-01 00:00:00,DR,Flood,FLOOD,1954-07-01 00:00:00,1954-07-01 00:00:00,1959-07-01,0,NaN,48,
18,20,No,Yes,Yes,Yes,SD,1954-07-31 00:00:00,DR,Flood,FLOOD,1954-07-31 00:00:00,1954-07-31 00:00:00,1956-02-01,0,NaN,46,


In [49]:
df['st'] = df['Location State Code'].map(lambda x: str(x)).apply(lambda x: x.zfill(2))
df['co'] = df['Location County Code'].map(lambda x: str(x)).apply(lambda x: x.zfill(3))

#add id column (fips) and test
df['fips'] = df['st'] + df['co']
#df['fips][67]

Timestamp('2018-04-15 00:00:00')

# FemaWebDisasterSummaries
contain the disaster number and total amt obligated
disaster nos. 1239-5238 (declared 1998 - 2018)
NO location info, but some financial summaries

http://www.fema.gov/api/open/v1/FemaWebDisasterSummaries.csv




In [32]:
df=pd.read_csv('fema/FemaWebDisasterSummaries.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2760 entries, 0 to 2759
Data columns (total 12 columns):
disasterNumber                2760 non-null int64
totalNumberIaApproved         363 non-null float64
totalAmountIhpApproved        363 non-null float64
totalAmountHaApproved         363 non-null float64
totalAmountOnaApproved        361 non-null float64
totalObligatedAmountPa        2198 non-null float64
totalObligatedAmountCatAb     2004 non-null float64
totalObligatedAmountCatC2g    0 non-null float64
paLoadDate                    2204 non-null object
iaLoadDate                    363 non-null object
hash                          2760 non-null object
lastRefresh                   2760 non-null object
dtypes: float64(7), int64(1), object(4)
memory usage: 258.8+ KB


# Disaster Areas
Place Code in the above sheet is a FEMA-specific data type, 99 followed by 3-digit FIPS code for county. 
disaster numbers 91 (1983) through 4366

API guide:
https://www.fema.gov/openfema-api-documentation

https://www.fema.gov/api/open/v1/DisasterDeclarationsSummaries?$filter=incidentType%20eq%20%27Flood%27

https://www.fema.gov/api/open/v1/DisasterDeclarationsSummaries?$filter=state%20eq%20%27VA%27%20or%20state%20eq%20%27NY%27

# FemaWebDisasterSummaries
 is the category that aggregates some of the expenditures for PA and IA.
The FEMA API only allows you to load 1000 rows at a time, but the csv file is also available at

https://www.fema.gov/openfema-dataset-fema-web-disaster-summaries-v1 (2760 rows)

Using the url
https://www.fema.gov/api/open/v1/FemaWebDisasterSummaries?$inlinecount=allpages

gives 2767 as the count. So we can add ?$skip=1000 to the url three times, concat the dfs, and get the (latest) 7 rows.

In [ ]:
#load json object using requests. Since there are 2767 rows total, we make 3 dataframes adding ?$skip=1000 and then ?$skip=2000 to the url, or stagger these somehow and then drop the dupes. 
myurl = 'https://www.fema.gov/api/open/v1/FemaWebDisasterSummaries?$skip=1800'

d = requests.get(myurl).json()
#load DataFrame from the appropriate key ('FemaWebDisasterSummaries')
df3 = json_normalize(d['FemaWebDisasterSummaries'])


In [60]:
df = pd.concat([df1,df2,df3])
df = df.drop_duplicates()
df.to_excel('fema/FemaWebDisasterSummaries.xls',index=False)

In [5]:
summ = pd.read_excel('fema/FemaWebDisasterSummaries.xls')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2767 entries, 0 to 2766
Data columns (total 12 columns):
disasterNumber               2767 non-null int64
hash                         2767 non-null object
iaLoadDate                   363 non-null object
id                           2767 non-null object
lastRefresh                  2767 non-null object
paLoadDate                   2211 non-null object
totalAmountHaApproved        363 non-null float64
totalAmountIhpApproved       363 non-null float64
totalAmountOnaApproved       361 non-null float64
totalNumberIaApproved        363 non-null float64
totalObligatedAmountCatAb    2008 non-null float64
totalObligatedAmountPa       2205 non-null float64
dtypes: float64(6), int64(1), object(5)
memory usage: 259.5+ KB


# Explanation of fields
from https://www.fema.gov/openfema-dataset-fema-web-disaster-summaries-v1

totalNumberIaApproved,Total Number IA Approved,number
The number of disaster assistance applications that were approved for Individual Assistance (IA)

totalAmountIhpApproved,Total Amount IHP Approved,number
The total amount approved for the Individual and Households Program (IHP). To learn more about the programs within IHP, please read the IHP Unified Guidance: https://www.fema.gov/media-library/assets/documents/124228 *THIS APPEARS TO BE THE LARGER NUMBER, WHERE IT IS GIVEN, and the SUM OF THE TWO FOLLOWING*

totalAmountHaApproved,Total Amount HA Approved,number
The total amount approved for Housing Assistance (HA) from the Individual and Households Program (IHP). To learn more about the programs within IHP, please read the IHP Unified Guidance: https://www.fema.gov/media-library/assets/documents/124228

totalAmountOnaApproved,Total Amount ONA Approved,number
The total amount approved for Other Needs Assistance (ONA) from the Individual and Households Program (IHP). To learn more about the programs within IHP, please read the IHP Unified Guidance: https://www.fema.gov/media-library/assets/documents/124228

totalObligatedAmountPa,Total Obligated Amount PA,number
The Public Assistance grant funding available to the grantee (State), for sub-grantee approved Project Worksheets

totalObligatedAmountCatAb,Total Obligated Amount Category A and B,number
The Public Assistance grant funding available to the grantee (State), for sub-grantee approved Project Worksheets in categories A and B, or Emergency Work. Category A: Debris removal; Category B: Emergency protective measures. To learn more about FEMA categories of work, visit: https://www.fema.gov/media-library/assets/documents/90743

totalObligatedAmountCatC2G,Total Obligated Amount Catories C to G,number
The Public Assistance grant funding available to the grantee (State), for sub-grantee approved Project Worksheets in categories C through G, or Permanent Work. Category C: Roads and bridges; Category D: Water control facilities; Category E: Public buildings and contents; Category F: Public utilities; Category G: Parks, recreational, and other facilities. To learn more about FEMA categories of work, visit: https://www.fema.gov/media-library/assets/documents/90743

paLoadDate,PA Load Date,date
The date the PA (Public Assistance) data was updated by NEMIS.


totalObligatedAmountPa + totalAmountIhpApproved may cover all BUT Hazard Mitigation


In [61]:
df = pd.read_excel('fema/FemaWebDisasterSummaries.xls')

In [67]:
#let's break this down to disasters related to landscape function
soil = ['Severe Storm(s)', 'Mud/Landslide', 'Flood',
       'Dam/Levee Break', 'Drought']
df = df[df['incidentType'].isin(soil)]
df.incidentType.unique()

array(['Severe Storm(s)', 'Mud/Landslide', 'Flood', 'Dam/Levee Break',
       'Drought'], dtype=object)

# Individual Assistance

The IA data on FemaWebDisasterSummaries is extremely sparse for Individual Assistance (IA), only 363 rows with figures. More complete data is available from

https://www.fema.gov/openfema-dataset-housing-assistance-data-renters-v1 and
https://www.fema.gov/openfema-dataset-housing-assistance-data-owners-v1

from these we can get csv files that contain most available data and supplement more recent entries via the API

https://fema.gov/api/open/v1/HousingAssistanceOwners

86911 rows for Owners
62481 rows for Renters

However, the earliest disaster info reported is 1539, tropical storm BONNIE, 2004. 



In [3]:
renters = pd.read_csv('fema/HousingAssistanceOwners.csv')
renters.disasterNumber.min()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1539

In [27]:
#load json object using requests. Since there are 86911 rows total, we make 86 dataframes adding ?$skip=1000 and then incrementing this by 1000. 
myurl = 'https://www.fema.gov/api/open/v1/HousingAssistanceRenters?$skip=62240'

d = requests.get(myurl).json()
#load DataFrame from the appropriate key
df = json_normalize(d['HousingAssistanceRenters'])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 23 columns):
approvedBetween10001And25000    241 non-null int64
approvedBetween1And10000        241 non-null int64
approvedBetween25001AndMax      241 non-null int64
approvedForFemaAssistance       241 non-null int64
city                            241 non-null object
county                          241 non-null object
disasterNumber                  241 non-null int64
hash                            241 non-null object
id                              241 non-null object
lastRefresh                     241 non-null object
otherNeedsAmount                241 non-null float64
rentalAmount                    241 non-null float64
repairReplaceAmount             241 non-null float64
state                           241 non-null object
totalApprovedIhpAmount          241 non-null float64
totalInspected                  241 non-null int64
totalInspectedWithNoDamage      241 non-null int64
totalMaxGrants

In [28]:
df.drop('id',axis=1, inplace=True)
df2 = pd.concat([df0,df])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62481 entries, 0 to 240
Data columns (total 22 columns):
approvedBetween10001And25000    62481 non-null int64
approvedBetween1And10000        62481 non-null int64
approvedBetween25001AndMax      62481 non-null int64
approvedForFemaAssistance       62481 non-null int64
city                            62475 non-null object
county                          62481 non-null object
disasterNumber                  62481 non-null int64
hash                            62481 non-null object
lastRefresh                     62481 non-null object
otherNeedsAmount                62481 non-null float64
rentalAmount                    62481 non-null float64
repairReplaceAmount             62481 non-null float64
state                           62481 non-null object
totalApprovedIhpAmount          62481 non-null float64
totalInspected                  62481 non-null int64
totalInspectedWithNoDamage      62481 non-null int64
totalMaxGrants                  

In [46]:
df2 = df2[['city','county','disasterNumber','state','totalApprovedIhpAmount','zipCode']]



In [50]:
g = df2.groupby(['state','disasterNumber','county']).agg({'totalApprovedIhpAmount': np.sum})
g.head(40)

totalApprovedIhpAmount
state disasterNumber county                                                                    
AK    1843           Alaska Gateway Regional Educational Attendance ...                 67061.0
                     Kuspuk Regional Educational Attendance Area                       103164.0
                     Lower Yukon Regional Educational Attendance Area                  295999.0
                     Yukon Flats Regional Educational Attendance Area                  131785.0
                     Yukon Koyukuk Regional Educational Attendance Area                 34349.0
                     Yupiit Regional Educational Attendance Area                       175669.0
      4122           Lower Yukon Regional Educational Attendance Area                  194372.0
                     Yukon Flats Regional Educational Attendance Area                   69454.0
                     Yukon Koyukuk Regional Educational Attendance Area                251270.0
AL    1549           Autauga (County)                                                  169653.0
                     Baldwin (County)                                                 6871719.0
                     Barbour (County)                                                   60936.0
                     Bibb (County)                                                      34629.0
                     Blount (County)                                                     1637.0
                     Bullock (County)                                                  443572.0
                     Butler (County)                                                   425689.0
                     Calhoun (County)                                                    1867.0
                     Chambers (County)                                                   2480.0
                     Chilton (County)                                                   27334.0
                     Choctaw (County)                                                  126546.0
                     Clarke (County)                                                   655559.0
                     Coffee (County)                                                    69555.0
                     Colbert (County)                                                       0.0
                     Conecuh (County)                                                  369054.0
                     Coosa (County)                                                      6671.0
                     Covington (County)                                                276207.0
                     Crenshaw (County)                                                  61432.0
                     Dale (County)                                                      16990.0
                     Dallas (County)                                                  1691584.0
                     DeKalb (County)                                                      349.0
                     Elmore (County)                                                   198254.0
                     Escambia (County)                                                3605920.0
                     Etowah (County)                                                     1907.0
                     Franklin (County)                                                  20264.0
                     Geneva (County)                                                    46323.0
                     Greene (County)                                                   215866.0
                     Hale (County)                                                     149337.0
                     Henry (County)                                                      1945.0
                     Houston (County)                                                   74184.0
                     Jackson (County)                                                       0.0

# Public Assistance
https://www.fema.gov/openfema-dataset-public-assistance-funded-projects-summaries-v1

API

https://www.fema.gov/api/open/v1/PublicAssistanceFundedProjectsSummaries?$inlinecount=allpages

shows 158869

In [14]:
pa1 = pd.read_csv('fema/PublicAssistanceFundedProjectsSummaries.csv')
pa1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158164 entries, 0 to 158163
Data columns (total 11 columns):
disasterNumber            158164 non-null int64
declarationDate           158164 non-null object
incidentType              158164 non-null object
state                     158164 non-null object
county                    152067 non-null object
applicantName             158164 non-null object
educationApplicant        158164 non-null int64
numberOfProjects          158164 non-null int64
federalObligatedAmount    158158 non-null float64
hash                      158164 non-null object
lastRefresh               158164 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 13.3+ MB


In [16]:
#load json object using requests. Since there are 158869 rows total, and we have 158164 rows in the csv, we only want the most recent records
myurl = 'https://www.fema.gov/api/open/v1/PublicAssistanceFundedProjectsSummaries?$skip=158164'

d = requests.get(myurl).json()
#load DataFrame from the appropriate key
pa2 = json_normalize(d['PublicAssistanceFundedProjectsSummaries'])
pa2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 12 columns):
applicantName             705 non-null object
county                    705 non-null object
declarationDate           705 non-null object
disasterNumber            705 non-null int64
educationApplicant        705 non-null bool
federalObligatedAmount    705 non-null float64
hash                      705 non-null object
id                        705 non-null object
incidentType              705 non-null object
lastRefresh               705 non-null object
numberOfProjects          705 non-null int64
state                     705 non-null object
dtypes: bool(1), float64(1), int64(2), object(8)
memory usage: 61.4+ KB


In [17]:
#the id column is not used in the csv so we drop it and concatenate rows
pa2.drop('id',axis=1, inplace=True)
pa3 = pd.concat([pa1,pa2])
pa3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158869 entries, 0 to 704
Data columns (total 11 columns):
applicantName             158869 non-null object
county                    152772 non-null object
declarationDate           158869 non-null object
disasterNumber            158869 non-null int64
educationApplicant        158869 non-null int64
federalObligatedAmount    158863 non-null float64
hash                      158869 non-null object
incidentType              158869 non-null object
lastRefresh               158869 non-null object
numberOfProjects          158869 non-null int64
state                     158869 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 14.5+ MB


In [21]:
pa3.to_csv('fema/pa.csv', index=False)

# Hazard Mitigation Projects

This is an overview. The program started in 1989.
https://www.fema.gov/hazard-mitigation-grant-program

And this seems a fairly complete spreadsheet
https://www.fema.gov/media-library-data/1515165332729-27d87a97bf796399295db276ff53a7fb/FEMAHazardMitigation_1.5.2018.xlsx

don't use the Properties database:
https://www.fema.gov/api/open/v1/HazardMitigationAssistanceMitigatedProperties?$inlinecount=allpages shows
118987 records in the database!

HazardMitigationAssistanceProjects.csv only starts with 1239

25735 records in the API 
https://www.fema.gov/api/open/v1/HazardMitigationAssistanceProjects?$inlinecount=allpages

many records DO NOT have a disaster number

In [ ]:
hm = pd.read_excel('fema/FEMAHazardMitigation_1.5.2018.xlsx','FEMA_HAZ_MIT',skiprows=2)

In [12]:
soilRelated = ['Flood','Fire','Drought', 'Mud/Landslide','Dam/Levee Break']
hm = hm[hm['Incident Type'].isin(soilRelated)]
hm.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3280 entries, 0 to 23678
Data columns (total 16 columns):
Region                     3280 non-null int64
State                      3280 non-null object
Disaster Number            3280 non-null int64
Declaration Date           3280 non-null datetime64[ns]
Incident Type              3280 non-null object
Disaster Title             3280 non-null object
Project Number             3280 non-null object
Project Type               3275 non-null object
Project Title              3280 non-null object
Project Description        2957 non-null object
Project Counties           3268 non-null object
Status                     3280 non-null object
Subgrantee                 3277 non-null object
Project Amount             3280 non-null float64
Federal Share Obligated    3280 non-null int64
Cost Share Percentage      3280 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(3), object(10)
memory usage: 435.6+ KB


In [18]:
#How much is statewide?
a = hm[hm['Project Counties']=='STATEWIDE']['Project Amount'].sum()
b = hm['Project Amount'].sum()
print(a/b)

0.08037755764892623


In [14]:
#g = hm.groupby(['Incident Type']).agg({'Project Amount': np.sum})
hm.to_excel('fema/HMsoilRelated.xls',index=False)